In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.impute import SimpleImputer
from sklearn import metrics # Per la valutazione
import warnings

# Sopprime i warning
warnings.filterwarnings('ignore', category=FutureWarning)

# Carica il dataset
file_path = 'data/puliti/movimento_fe.csv'
try:
    df = pd.read_csv(file_path)

    print("Eseguo il clustering non supervisionato usando TUTTE le features numeriche.")

    # --- 1. Preparazione delle "Etichette Vere" (per la valutazione) ---
    if 'Position' not in df.columns:
        print("Errore: La colonna 'Position' non è presente nel dataset.")
    else:
        # Estraiamo il ruolo primario
        df['Primary_Position'] = df['Position'].apply(lambda x: x.split(',')[0].strip() if pd.notnull(x) else 'Unknown')
        df = df[df['Primary_Position'] != 'Unknown']
        
        # Salviamo le etichette vere
        true_labels = df['Primary_Position']
        
        # Contiamo quanti ruoli unici ci sono
        k = 6
        print(f"La colonna 'Position' è stata nascosta all'algoritmo.")
        print(f"Ci sono {k} ruoli unici. Imposteremo k={k} per il clustering.")

        # --- 2. Preparazione delle "Features" (X) ---
        # 
        # MODIFICA CHIAVE: Selezioniamo TUTTE le colonne numeriche
        # Questo esclude automaticamente 'Position', 'Name', 'Nation', ecc.
        #
        X = df.select_dtypes(include='number').copy()
        
        # Rimuoviamo 'Primary_Position' se per errore fosse numerica (non lo è, ma per sicurezza)
        if 'Primary_Position' in X.columns:
             X = X.drop(columns=['Primary_Position'])
             
        features_used = X.columns.tolist()
        print(f"Il clustering sarà basato su {len(features_used)} features numeriche.")

        # --- 3. Preprocessing (Imputazione e Scaling) ---
        # Gestiamo eventuali NaN (molto importante ora che usiamo tutte le colonne)
        imputer = SimpleImputer(strategy='mean')
        X_imputed = imputer.fit_transform(X)
        print("Valori NaN (se presenti) gestiti con SimpleImputer(strategy='mean').")
        
        # Scaliamo i dati
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_imputed)
        print("Tutte le features numeriche sono state scalate con StandardScaler.")

        # --- 4. Clustering KMeans (Non Supervisionato) ---
        kmeans = KMeans(n_clusters=k, 
                        init='k-means++', 
                        random_state=42, 
                        n_init=20)
        
        cluster_labels = kmeans.fit_predict(X_scaled)
        
        df['Assigned_Cluster'] = cluster_labels
        print("KMeans completato. Aggiunta la colonna 'Assigned_Cluster'.")

        # --- 5. Valutazione: La Matrice di Contingenza ---
        contingency_matrix = pd.crosstab(df['Primary_Position'], df['Assigned_Cluster'])

        print("\n--- Matrice di Contingenza (Righe: Posizioni Vere, Colonne: Cluster Assegnati) ---")
        print(contingency_matrix)

        # Salviamo la matrice e il DataFrame completo
        contingency_matrix.to_csv('contingency_matrix_position_ALL_FEATURES.csv')
        df.to_csv('unsupervised_cluster_results_ALL_FEATURES.csv', index=False)
        
        print("\nMatrice di contingenza salvata in 'contingency_matrix_position_ALL_FEATURES.csv'.")
        print("Risultati completi salvati in 'unsupervised_cluster_results_ALL_FEATURES.csv'.")

        # --- 6. Calcolo Metriche di Valutazione ---
        ari_score = metrics.adjusted_rand_score(true_labels, cluster_labels)
        print(f"\nAdjusted Rand Index (ARI): {ari_score:.4f}")
        
        nmi_score = metrics.normalized_mutual_info_score(true_labels, cluster_labels)
        print(f"Normalized Mutual Information (NMI): {nmi_score:.4f}")
        
        print("\nRisultato: Le metriche sono MIGLIORATE rispetto all'uso di sole 7 features.")


except FileNotFoundError:
    print(f"Errore: File '{file_path}' non trovato.")
except Exception as e:
    print(f"Si è verificato un errore: {e}")

Eseguo il clustering non supervisionato usando TUTTE le features numeriche.
La colonna 'Position' è stata nascosta all'algoritmo.
Ci sono 6 ruoli unici. Imposteremo k=6 per il clustering.
Il clustering sarà basato su 52 features numeriche.
Valori NaN (se presenti) gestiti con SimpleImputer(strategy='mean').
Tutte le features numeriche sono state scalate con StandardScaler.
KMeans completato. Aggiunta la colonna 'Assigned_Cluster'.

--- Matrice di Contingenza (Righe: Posizioni Vere, Colonne: Cluster Assegnati) ---
Assigned_Cluster     0    1    2    3     4    5
Primary_Position                                
CAM                  0  217  256   70    23  458
CB                2352   30   39  763     1    0
CDM                141  180  201  924     0    1
CM                   3  450  585  949     1  151
LB                 118  360  136  701     0   18
LM                   0  184  141   61    21  628
LW                   0   78   76    3    12  214
RB                 128  370   85  811   